In [1]:
# framework Langchain


# within an llm

# limitation ----> context window

In [ ]:
! pip install langchain-community
! pip install pypdf

In [3]:
from langchain_community.document_loaders import PyPDFLoader

path = "budget_speech.pdf"

loader = PyPDFLoader(path)

In [4]:
docs = loader.load()

In [22]:
docs[2].page_content

'CONTENTS \n \nPART – A \n Page No. \nIntroduction 1 \nBudget Theme 1 \nAgriculture as the 1st engine 3 \nMSMEs as the 2nd engine 6 \nInvestment as the 3rd engine 8 \nA. Investing in People 8 \nB. Investing in the Economy 10 \nC. Investing in Innovation 14 \nExports as the 4th engine 15 \nReforms as the Fuel 16 \nFiscal Policy 18 \n \n \nPART – B \nIndirect taxes 20 \nDirect Taxes  23 \n \nAnnexure to Part-A 29 \nAnnexure to Part-B 31'

In [6]:
from langchain_text_splitters import CharacterTextSplitter

In [7]:
text_splitter = CharacterTextSplitter(chunk_size = 1000)

In [8]:
chunks = text_splitter.split_documents(docs)

In [9]:
print(chunks[0].page_content)

GOVERNMENT OF INDIA
BUDGET 2025-2026
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1,  2025


In [10]:
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.6 MB/s eta 0:00:00


In [11]:
! pip install sentence-transformers

In [12]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [13]:
embeddinga_model_name = "all-MiniLM-L6-v2"

In [14]:
embeddings = HuggingFaceEmbeddings(model_name=embeddinga_model_name)

/tmp/ipython-input-402301105.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddinga_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
vector_db = FAISS.from_documents(chunks,embeddings)

In [16]:
result  = vector_db.similarity_search("Building Rural Prosperity and Resilience")

In [20]:
result1  = vector_db.similarity_search("what is the budget allocation for health care")

In [21]:
print(result1[0].page_content)

19  
 
Budget Estimates 2025-26 
112. Coming to 2025 -26, the total receipts other than  borrowings and the 
total expenditure are estimated at ` 34.96 lakh crore and ` 50.65 lakh crore 
respectively. The net tax receipts are estimated at ` 28.37 lakh crore. 
113. The fiscal deficit is estimated to be 4.4 per cent of GDP. 
114. To finance the fiscal deficit, the net market borrowings from dated 
securities are estimated at ` 11.54 lakh crore. The balance financing is expected 
to come from small savings and other sources. The gross market borrowings 
are estimated at ` 14.82 lakh crore. 
I will now move to Part B.


In [17]:
print(result[0].page_content)

2  
 
grown in this period. We see the next five years as a unique opportunity to 
realize ‘Sabka Vikas’, stimulating balanced growth of all regions. 
5. The great Telugu poet and playwright Gurajada Appa Rao had said, 
‘Desamante Matti Kaadoi, Desamante Manushuloi’; meaning, ‘A country is not 
just its soil, a country is its people.’ In line with this, for us, Viksit Bharat, 
encompasses: 
a) zero-poverty; 
b) hundred per cent good quality school education;   
c) access to high-quality, affordable, and comprehensive healthcare; 
d) hundred per cent skilled labour with meaningful employment; 
e) seventy per cent women in economic activities; and 
f) farmers making our country the ‘food basket of the world’.  
6. In this Budget, the proposed development measures span ten broad 
areas focusing on Garib, Youth, Annadata and Nari. 
1) Spurring Agricultural Growth and Productivity; 
2) Building Rural Prosperity and Resilience;  
3) Taking Everyone Together on an Inclusive Growth path; 
4) B

In [18]:
# Augmentation

In [19]:
from groq import Groq

ModuleNotFoundError: No module named 'groq'

In [ ]:
key = "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"
modelused = "llama-3.1-8b-instant"

In [ ]:
prompt = """
You are a smart chatbot, you need to respond to user questions only by refering to the data present within the below knowledge base, dont give any reference of the chunk which you are refering to just return a weel structured response as the answer to the user question.
"""

In [ ]:
def chatbot(message):
    result  = vector_db.similarity_search(message,k=3)
    context = []
    for i in result:
        context.append(f"Chunk: {i.page_content}")
    client = Groq(api_key=key)
    finalprompt = f"{prompt}\n\nKnowledge Base: {context}"
    print(finalprompt)
    completion = client.chat.completions.create(
        model=modelused,
        messages=[
            {
                "role": "system",
                "content": finalprompt
            },
            {
                "role": "user",
                "content": message
            }
        ],
        temperature=0,
        max_completion_tokens=1024
    )

    return completion.choices[0].message.content


In [ ]:
chatbot("best car to buy in 2025?")

In [23]:
from langchain_community.document_loaders import CSVLoader
path = "ncr_ride_bookings.csv"
loader = CSVLoader(path)

In [24]:
docs = loader.load()

In [ ]:
docs

In [27]:
docs[0]

Document(metadata={'source': 'ncr_ride_bookings.csv', 'row': 0}, page_content='Date: 2024-03-23\nTime: 12:29:38\nBooking ID: "CNR5884300"\nBooking Status: No Driver Found\nCustomer ID: "CID1982111"\nVehicle Type: eBike\nPickup Location: Palam Vihar\nDrop Location: Jhilmil\nAvg VTAT: null\nAvg CTAT: null\nCancelled Rides by Customer: null\nReason for cancelling by Customer: null\nCancelled Rides by Driver: null\nDriver Cancellation Reason: null\nIncomplete Rides: null\nIncomplete Rides Reason: null\nBooking Value: null\nRide Distance: null\nDriver Ratings: null\nCustomer Rating: null\nPayment Method: null')

In [28]:
from langchain_text_splitters import CharacterTextSplitter

# from langchain_text_splitters import RecursiveCharacterTextSplitter - Home work - 11/12/2025 - weekday

In [29]:
text_splitter = CharacterTextSplitter(chunk_size = 1500)

In [30]:
chunks = text_splitter.split_documents(docs)

In [ ]:
print(chunks[0])

In [ ]:
print(chunks[1])

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings


In [35]:
embedding_model_name = "all-MiniLM-L6-v2"

In [36]:
embeddings = HuggingFaceBgeEmbeddings(model_name= embeddinga_model_name)

/tmp/ipython-input-2832077859.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name= embeddinga_model_name)


In [ ]:
vector_db = FAISS.from_documents(chunks,embeddings)

In [ ]:
result = vector_db.similarity_search("what is the pickup location for booking id CNR5884300" )